# Least-squares fitting to estimate 3D rotation vector from 2D images 

This script estimates the rotational axis, for solid body rotation about an aribtrary axis in 3D, that gives rise to an observed displacement field computed from 2D images.  

Input list of point locations and displacement fields:
{(r_x, r_z)}_i for i in N

Params: (w_x, w_y, w_z)

Model = A*w, where A = [0 r_z -+ (R - r_x^2 -r_z^2)^(1/2); 1 -r_x 0]_{i} for i in N

Data: {(v_x, v_z)}_i for i in N


In [65]:
import numpy as np
import cv2
import pandas as pd
# Load the data
file = 'C:/Users/Deepak/Dropbox/ActiveMassTransport_Vorticella_SinkingAggregates/RotationalAnalysis/test_images_2020-12-08 01-41/RotationAnalysisTracks/RotationTrack.csv'

R = 448/2.0 # Aggregate size for test images
df = pd.read_csv(file)

nTracks = int(max(df['feature ID']+1))

print(nTracks)

nTimepoints = int(len(df['Time'])/nTracks)
print(nTimepoints)

# generate a dict containing time series for each track
df_tracks = {ii : df.loc[df['feature ID']==ii] for ii in range(nTracks)}

df_tracks

3
49


{0:     Unnamed: 0  feature ID  Time  feature centroid X  feature centroid Z  \
 0            0         0.0   0.0          366.519562          432.009918   
 1            1         0.0   1.0          367.376923          432.075989   
 2            2         0.0   2.0          368.208679          432.149658   
 3            3         0.0   3.0          369.013794          432.228119   
 4            4         0.0   4.0          369.798937          431.336826   
 5            5         0.0   5.0          372.657299          431.675308   
 6            6         0.0   6.0          372.610842          431.314482   
 7            7         0.0   7.0          373.432402          430.797300   
 8            8         0.0   8.0          374.223326          430.944792   
 9            9         0.0   9.0          375.011253          430.101768   
 10          10         0.0  10.0          375.923502          429.122875   
 11          11         0.0  11.0          377.752165          428.696379

{0:      Unnamed: 0  track ID       Time  centroid X  centroid Y
 0             0       0.0  60.008025  870.571554  421.985855
 1             1       0.0  60.241555  876.617502  423.788601
 2             2       0.0  60.481319  886.402588  426.628815
 3             3       0.0  60.745621  890.124207  428.237000
 4             4       0.0  61.008494  888.877451  430.029232
 ..          ...       ...        ...         ...         ...
 111         111       0.0  88.511613  667.526155  341.107401
 112         112       0.0  88.809424  667.764559  338.584146
 113         113       0.0  89.058303  663.025606  339.066599
 114         114       0.0  89.304017  657.455416  337.413370
 115         115       0.0  89.535101  658.459261  340.320475
 
 [116 rows x 5 columns],
 1:      Unnamed: 0  track ID       Time  centroid X  centroid Y
 116           0       1.0  60.008025  680.536865  327.551697
 117           1       1.0  60.241555  685.879211  328.551239
 118           2       1.0  60.481319

In [74]:
def residual(params, data):
    
    array_A = np.zeros((2,3))

    for ii in range(nTracks):
        
        centroids_x, centroids_z = np.array(df_tracks[ii]['feature centroid X']), np.array(df_tracks[ii]['feature centroid Z'])
        r_x, r_z = centroids_x[time_index], centroids_z[time_index]
        array_A[2*ii,:] = [0, r_z, (R- r_x**2 -r_z**2)**(1/2)]
        array_A[2*ii+1,:] = [1, -r_z, 0]
    
    omega_vect = np.array([params['omega_x'], params['omega_y'], params['omega_z']])
    model = np.matmul(array_A, omega_vect)
    
    print(model.shape)
    
    return model - data
    


# Calculate centroid displacements for each track
centroids_x = {}
centroids_z = {}
centroids_disp_x = {}
centroids_disp_z = {}

for ii in range(nTracks):
    centroids_x[ii], centroids_z[ii] = np.array(df_tracks[ii]['feature centroid X']), np.array(df_tracks[ii]['feature centroid Z'])
    sphere_centroids_x, sphere_centroids_z = np.array(df_tracks[ii]['sphere centroid X']), np.array(df_tracks[ii]['sphere centroid Z'])

    centroids_disp_x[ii] = centroids_x[ii][1:]-centroids_x[ii][:-1]
    centroids_disp_z[ii] = centroids_z[ii][1:]-centroids_z[ii][:-1]
    

for time_index in range(nTimepoints-1):
    array_A = np.zeros((2*nTracks,3))
    data = np.zeros(2*nTracks)
    print(time_index)
    for ii in range(nTracks):
        r_x, r_z = centroids_x[ii][time_index] - sphere_centroids_x[time_index], centroids_z[ii][time_index] - sphere_centroids_z[time_index]
        array_A[2*ii,:] = [0, r_z, (R**2- r_x**2 -r_z**2)**(1/2)]
        array_A[2*ii+1,:] = [1, -r_z, 0]
        
        data[2*ii] = centroids_disp_x[ii][time_index]
        data[2*ii+1] = centroids_disp_z[ii][time_index]

    
#     print('Time: {} \n Matrix: {}'.format(df['Time'][time_index], array_A))
#     print(array_A.shape)
    omega_vect = np.array([1,1,1])
    model = np.matmul(array_A, omega_vect)
    
    print(data)
    print(model)

0
[ 0.85736084  0.06607056 -0.67304993 -0.01420593  0.30054665 -0.17619228]
[ 300.3333331  -111.00991821   27.49373783  136.49343872   93.85542497
  101.49619865]
1
[ 0.83175659  0.07366943 -2.07113647  2.43054199 -1.53774452 -0.49870586]
[ 300.14618828 -111.07598877   27.16674652  136.50764465   93.51351662
  101.67239094]
2
[ 0.80511475  0.07846069 -0.66596985  0.03460693 -0.92767334 -0.01409912]
[ 299.96440048 -111.1496582    30.65582894  134.07710266   93.13198614
  102.1710968 ]
3
[ 0.7851429  -0.89129257 -2.9896698   0.85324097 -0.92092896 -0.01071167]
[ 299.78743573 -111.2281189    30.41101687  134.04249573   93.33147396
  102.18519592]
4
[ 2.8583622   0.3384819  -0.53526306  0.87200928 -0.91885376 -0.84516907]
[ 299.22023962 -110.33682632   30.53957709  133.18925476   93.5298995
  102.19590759]
5
[-0.04645729 -0.36082649 -0.60353088  0.8690033  -0.91989708  0.00410748]
[ 298.57590852 -110.67530823   31.85667446  132.31724548   92.45330488
  103.04107666]
6
[ 0.82155991 -0.51718

In [75]:
model.shape

(6,)